In [1]:
# Установка transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 35.1 MB/s 
     |████████████████████████████████| 163 kB 70.8 MB/s 
     |████████████████████████████████| 6.6 MB 54.1 MB/s 


In [2]:
#Установка catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.2 MB/s 


In [3]:
import pandas as pd
from itertools import groupby

In [4]:
#import необходимых модулей

import pandas as pd
import yaml
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.utils.class_weight import compute_class_weight

import numpy as np
from datetime import datetime, timedelta

from catboost import CatBoostClassifier, Pool

In [5]:
#Считывание данных в DataFrame 
train= pd.read_csv('train_net_2.csv', sep=';', index_col=None)
test= pd.read_csv('test_net_2.csv', sep=';', index_col=None)


In [6]:
train

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,team_id,game_id,Период_1_всего,Период_2_всего,Период_3_всего,...,Категория 2 средняя оценка,Категория 2 среднее место,Категория 3 средняя оценка,Категория 3 среднее место,Категория 4 средняя оценка,Категория 4 среднее место,Категория 5 средняя оценка,Категория 5 среднее место,Категория 6 средняя оценка,Категория 6 среднее место
0,10884,4,4,4,4,1440,202,7,14,28,...,-5.0,5.0,78.2,4.2,25.6,4.0,21.0,2.2,118.8,5.0
1,10106,5,4,5,5,1536,211,21,21,12,...,24.0,3.0,93.4,2.8,32.0,3.0,22.6,2.2,182.8,2.8
2,10438,6,5,5,5,1742,242,62,42,58,...,12.0,3.4,76.6,2.6,31.6,2.4,29.2,2.2,160.4,3.0
3,10130,5,4,5,5,1272,184,24,24,33,...,4.8,3.8,73.2,3.8,27.8,4.2,21.4,1.6,133.4,4.0
4,10667,4,5,5,5,1382,193,54,49,152,...,14.0,4.0,83.2,3.0,33.2,3.8,31.0,2.6,175.6,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,10563,5,4,5,4,1274,184,10,1,11,...,-5.4,4.4,69.8,4.6,25.4,4.4,18.2,3.6,107.2,4.8
676,10386,5,4,5,5,1526,210,1,14,24,...,21.6,2.6,94.2,1.8,30.8,3.0,12.4,5.2,166.4,2.8
677,10675,5,4,5,4,1670,235,4,14,20,...,17.2,2.6,79.8,3.4,27.2,3.4,16.2,2.4,146.6,2.6
678,10366,4,4,4,4,1808,248,10,44,45,...,21.6,2.8,85.0,2.4,32.8,2.4,20.0,3.2,168.4,2.4


In [7]:
test

,id,team_id,game_id,Период_1_всего,Период_2_всего,Период_3_всего,Период_4_всего,Analytical thinking_prognoz,Systemic thinking_prognoz,Adaptability_prognoz,...,Категория 2 средняя оценка,Категория 2 среднее место,Категория 3 средняя оценка,Категория 3 среднее место,Категория 4 средняя оценка,Категория 4 среднее место,Категория 5 средняя оценка,Категория 5 среднее место,Категория 6 средняя оценка,Категория 6 среднее место
0,10199,1912,262,2,2,3,2,4,4,4,...,11.2,3.8,77.4,3.4,30.2,3.8,21.4,2.8,142.4,3.8
1,10539,1458,204,1,9,12,13,4,4,5,...,-0.6,3.6,71.2,3.2,23.6,4.0,26.2,4.0,116.8,4.0
2,10174,1348,190,0,28,21,16,5,5,4,...,41.2,1.8,96.2,1.4,40.2,1.8,24.4,2.2,226.8,1.8
3,10465,1760,244,15,18,29,17,4,5,5,...,-3.8,4.2,84.4,2.0,26.4,4.2,22.8,3.0,125.2,3.6
4,10066,1260,183,7,23,16,6,5,4,4,...,-16.0,4.4,62.8,4.8,23.0,4.6,18.8,3.4,79.0,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,10433,1562,214,15,6,1,0,4,5,4,...,52.4,2.4,94.4,3.4,47.8,1.8,25.2,2.8,245.0,2.4
289,10893,1278,184,14,52,34,18,5,5,5,...,31.4,2.8,82.6,3.0,40.0,1.8,21.0,2.6,186.6,2.6
290,10909,1474,206,0,0,0,1,3,4,3,...,17.0,2.8,75.8,4.0,30.4,3.0,24.4,3.6,153.0,3.2
291,10889,1268,184,6,2,5,1,3,3,3,...,39.4,3.4,102.4,1.8,39.8,2.8,11.6,5.6,213.6,2.6


In [8]:
train.columns

Index(['id', 'Analytical thinking', 'Systemic thinking', 'Adaptability',
       'Focus', 'team_id', 'game_id', 'Период_1_всего', 'Период_2_всего',
       'Период_3_всего', 'Период_4_всего', 'Analytical thinking_prognoz',
       'Systemic thinking_prognoz', 'Adaptability_prognoz', 'Focus_prognoz',
       'Категория 1 средняя оценка', 'Категория 1 среднее место',
       'Категория 2 средняя оценка', 'Категория 2 среднее место',
       'Категория 3 средняя оценка', 'Категория 3 среднее место',
       'Категория 4 средняя оценка', 'Категория 4 среднее место',
       'Категория 5 средняя оценка', 'Категория 5 среднее место',
       'Категория 6 средняя оценка', 'Категория 6 среднее место'],
      dtype='object')

In [9]:
# выбираем поля для нейросети СatBoost по бинарной классификации

X_train=train[['team_id', 'game_id',  'Analytical thinking_prognoz',
       'Systemic thinking_prognoz', 'Adaptability_prognoz', 'Focus_prognoz','Период_1_всего', 'Период_2_всего',
       'Период_3_всего', 'Период_4_всего',
        'Категория 1 среднее место',
        'Категория 2 среднее место',
        'Категория 3 среднее место',
        'Категория 4 среднее место',
        'Категория 5 среднее место',
        'Категория 6 среднее место']]
y_train=train[['Analytical thinking']]


testnet=test[['team_id', 'game_id', 'Analytical thinking_prognoz',
       'Systemic thinking_prognoz', 'Adaptability_prognoz', 'Focus_prognoz','Период_1_всего', 'Период_2_всего',
       'Период_3_всего', 'Период_4_всего',
        'Категория 1 среднее место',
        'Категория 2 среднее место',
        'Категория 3 среднее место',
        'Категория 4 среднее место',
        'Категория 5 среднее место',
        'Категория 6 среднее место']]

In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   team_id                      680 non-null    int64  
 1   game_id                      680 non-null    int64  
 2   Analytical thinking_prognoz  680 non-null    int64  
 3   Systemic thinking_prognoz    680 non-null    int64  
 4   Adaptability_prognoz         680 non-null    int64  
 5   Focus_prognoz                680 non-null    int64  
 6   Период_1_всего               680 non-null    int64  
 7   Период_2_всего               680 non-null    int64  
 8   Период_3_всего               680 non-null    int64  
 9   Период_4_всего               680 non-null    int64  
 10  Категория 1 среднее место    680 non-null    float64
 11  Категория 2 среднее место    680 non-null    float64
 12  Категория 3 среднее место    680 non-null    float64
 13  Категория 4 среднее 

In [11]:
# Определяем веса для классов кластеров
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes = np.unique(y_train), y = np.ravel(y_train))
class_weights = dict(zip(classes, weights))
class_weights

{1: 34.0,
 3: 3.7777777777777777,
 4: 0.3261390887290168,
 5: 0.6507177033492823,
 6: 9.714285714285714}

In [12]:
#Обучаем 

pool_train_solution = Pool(X_train, y_train, cat_features = ['team_id', 'game_id','Analytical thinking_prognoz',
       'Systemic thinking_prognoz', 'Adaptability_prognoz', 'Focus_prognoz'])
pool_test_solution = Pool(testnet,cat_features = ['team_id', 'game_id','Analytical thinking_prognoz',
       'Systemic thinking_prognoz', 'Adaptability_prognoz', 'Focus_prognoz'])

model_test = CatBoostClassifier(iterations=100,  task_type='GPU',depth=12, verbose =True, random_seed=9,# learning_rate=0.015,
                               leaf_estimation_method= 'Gradient',bootstrap_type= 'Poisson',objective = 'MultiClass',eval_metric = 'Accuracy',loss_function='Recall', class_weights=class_weights)
model_test.fit(pool_train_solution)

0:	learn: 0.8716998	total: 166ms	remaining: 16.5s
1:	learn: 0.8746666	total: 283ms	remaining: 13.9s
2:	learn: 0.8893188	total: 400ms	remaining: 12.9s
3:	learn: 0.9012862	total: 476ms	remaining: 11.4s
4:	learn: 0.8988928	total: 552ms	remaining: 10.5s
5:	learn: 0.9031978	total: 644ms	remaining: 10.1s
6:	learn: 0.9075098	total: 719ms	remaining: 9.55s
7:	learn: 0.9031955	total: 796ms	remaining: 9.15s
8:	learn: 0.9036751	total: 876ms	remaining: 8.85s
9:	learn: 0.9012839	total: 951ms	remaining: 8.56s
10:	learn: 0.9031978	total: 1.03s	remaining: 8.38s
11:	learn: 0.8993655	total: 1.11s	remaining: 8.16s
12:	learn: 0.9017659	total: 1.19s	remaining: 7.96s
13:	learn: 0.9017636	total: 1.26s	remaining: 7.77s
14:	learn: 0.9012839	total: 1.34s	remaining: 7.61s
15:	learn: 0.8993701	total: 1.42s	remaining: 7.45s
16:	learn: 0.9017636	total: 1.5s	remaining: 7.3s
17:	learn: 0.9036797	total: 1.57s	remaining: 7.18s
18:	learn: 0.9041594	total: 1.7s	remaining: 7.24s
19:	learn: 0.9041594	total: 1.8s	remaining: 

In [13]:
# предварительный тест схождения на всем обучающем множестве
y_pred = model_test.predict(X_train)


In [14]:
print("Значение метрики recall на трейне : ", recall_score(y_train, y_pred, average='macro', zero_division=True))

Значение метрики recall на трейне :  0.9036797356373274


In [15]:
#Получение ответов
y_pred_solution = model_test.predict(pool_test_solution)

In [16]:
# Посмотрим предварительный прогноз
test['Analytical thinking_prognoz'].value_counts()

4    132
5    112
3     46
6      3
Name: Analytical thinking_prognoz, dtype: int64

In [17]:
# Заводим результаты в тест
test['Analytical thinking']=y_pred_solution
test['Analytical thinking'].value_counts()

4    141
5    111
3     38
6      3
Name: Analytical thinking, dtype: int64

In [18]:
test_itog=test[['id','Analytical thinking']]
test_itog

,id,Analytical thinking
0,10199,4
1,10539,4
2,10174,5
3,10465,4
4,10066,5
...,...,...
288,10433,4
289,10893,5
290,10909,3
291,10889,3


In [19]:
test_itog.to_csv('прогноз по Analytical thinking предварительный второй этап 100 9 c действиями без оценки.csv', sep=',', index=None)